In [1]:
import os
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
files = os.listdir("data")

In [3]:
files

['geschaeft2009_204.htm',
 'geschaeft2019_204.htm',
 'geschaeft2015_84.htm',
 'geschaeft2020_2.htm',
 'geschaeft2011_362.htm',
 'geschaeft2004_541.htm',
 'geschaeft2020_17.htm',
 'geschaeft1998_191.htm',
 'geschaeft2011_438.htm',
 'geschaeft1998_59.htm',
 'geschaeft2017_455.htm',
 'geschaeft2007_327.htm',
 'geschaeft2007_469.htm',
 'geschaeft2001_389.htm',
 'geschaeft2005_81.htm',
 'geschaeft2015_236.htm',
 'geschaeft2017_9.htm',
 'geschaeft2018_215.htm',
 'geschaeft2011_148.htm',
 'geschaeft1999_180.htm',
 'geschaeft2016_450.htm',
 'geschaeft1998_378.htm',
 'geschaeft2016_322.htm',
 'geschaeft2017_131.htm',
 'geschaeft2006_450.htm',
 'geschaeft2007_125.htm',
 'geschaeft2013_273.htm',
 'geschaeft1998_422.htm',
 'geschaeft2016_16.htm',
 'geschaeft2006_295.htm',
 'geschaeft2016_281.htm',
 'geschaeft2007_90.htm',
 'geschaeft2000_99.htm',
 'geschaeft2006_530.htm',
 'geschaeft2014_192.htm',
 'geschaeft2016_242.htm',
 'geschaeft2013_26.htm',
 'geschaeft2006_242.htm',
 'geschaeft2005_381.htm'

In [4]:
file = open("data/" + str(files[0]), "r")

In [5]:
text = file.read()

In [6]:
soup = BeautifulSoup(text, "html.parser")

In [7]:
soup

<html lang="de"><head><meta content="text/html;charset=utf-8" http-equiv="Content-type"/><meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/><meta content="Gemeinderat Zürich" name="author"/>
<meta content="" name="description"/>
<meta content="" name="keywords"/>
<title>
	Detailansicht Geschäft | Gemeinderat Zürich
</title><link href="/css/Normalize.css" rel="stylesheet"/><link href="/css/grz-theme/jquery-ui-1.10.4.custom.css" rel="stylesheet"/><link href="/css/GRZ.css" rel="stylesheet"/><link href="/css/lightbox.css" rel="stylesheet"/><link href="/css/Gemeinderat.css" rel="stylesheet"/><link href="/css/colorbox.css" rel="stylesheet"/><link href="/scripts/socialprivacy/stylesheets/socialshareprivacy.css" rel="stylesheet"/>
<script src="/scripts/jquery-1.11.0.js" type="text/javascript"></script>
<script src="/scripts/jquery-ui-1.10.4.custom.js" type="text/javascript"></script>
<script src="/scripts/jquery.colorbox-min.js" type="text/javascript"></script>
<script src="/script

### Schritt 1: Die Name und Partei lokalisieren

In [8]:
#Hierist der Erstunterzeichner drin:
soup.find_all("a")[23]

<a class="unterzeichnenderLink" href="/Mitglieder/Detailansicht-Mitglied?mid=93baa3d3-84f7-4c89-841b-b5ea7f4882a5" id="ContentPlaceHolderDefault_ContentPlaceHolderBody_ContentPlaceHolderDefault_ContentPlaceHolderContent_ctl00_GeschaeftControl_13_ErstUnterzeichnender">Hans Bachmann (FDP) </a>

In [9]:
#Prüfen wir das an einem anderen Beispiel mit zwei Unterzeichnenden:
file2 = open("data/geschaeft1999_433.htm", "r")
text2 = file2.read()
soup2 = BeautifulSoup(text2, "html.parser")
soup2.find_all("a")[23:25]

#bei mehreren ist das in Positionen 23,24 etc. drin > Problem: das ist nicht stabil, Positionen verschieben sich
# stattdessen mit div_Tag probeieren

FileNotFoundError: [Errno 2] No such file or directory: 'data/geschaeft1999_433.htm'

In [10]:
#Fazit: es die Namen sind alle in Div-Tag Nr. 24, und dort als a-Tags erfasst:
soup2.find_all("div")[24].find_all("a")

#Und so holt man sie raus:
soup2.find_all("div")[24].find_all("a")[0].text.strip()

NameError: name 'soup2' is not defined

In [11]:
# und das jetzt noch splitten: 
partei = soup2.find_all("div")[24].find_all("a")[0].text.strip().split(" (")[1][:-1]
partei

name = soup2.find_all("div")[24].find_all("a")[0].text.strip().split(" (")[0]
name
vorname = name.split(" ")[0]
vorname

NameError: name 'soup2' is not defined

### Schritt 2: Geschäftsnummer rauslesen

In [12]:
soup2.find_all("div")[22].find_all("span")[1].text

NameError: name 'soup2' is not defined

### Schritt 3: Departement rauslesen

In [13]:
soup2.find_all("div")[28].find_all("span")[2].text

NameError: name 'soup2' is not defined

In [14]:
#funktioniert nicht immer, deshlab vielleicht eher mit Regex

In [15]:
import re

try:
    regex_dep = r"((?<=Zuständiges Departement).*\n*.*)"
    dep_fragment = re.findall(regex_dep, str(soup2))[0]
    soup_dep = BeautifulSoup(dep_fragment, "html.parser")
    departement = soup_dep.text[1:]

except:
    departement = ""
    
departement

''

### Schritt 4: Datum rauslesen

In [16]:
soup2.find_all("div")[31].find_all("strong")[1].text

NameError: name 'soup2' is not defined

In [17]:
#Hier ist eingentlich alles drin, auch die Namen

In [18]:
date_junk = soup2.find_all("div")[31].find_all("strong")[1].text
date = date_junk.split(" vom ")

NameError: name 'soup2' is not defined

In [19]:
date

NameError: name 'date' is not defined

In [20]:
date = date[1][:-1].strip()

NameError: name 'date' is not defined

In [21]:
date

NameError: name 'date' is not defined

### Vorarbeiten 1 für den Loop: Stossen wir auf Fehler?

In [22]:
for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("data/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        print (file) #damit finden wir raus, wo es allenfalls Probleme gibt
    
        einreichender1 = soup.find_all("div")[24].find_all("a")[0].text.strip()
            
        #zuerst: wir wollen ja nur jene haben, die überhaupt einen Namen drin haben; 
        #Erkennungsmerkmal: die haben immer 2 Klammern drin
        if "(" and ")" in einreichender1:
            print(einreichender1)
            
#### Yes! So funktioniert das fehlerfrei.

geschaeft2009_204.htm
Hans Bachmann (FDP)
geschaeft2019_204.htm
Felix Moser (Grüne)
geschaeft2015_84.htm
Karin Rykart Sutter (Grüne)
geschaeft2020_2.htm
Marcel Bührig (Grüne)
geschaeft2011_362.htm
Alecs Recher (AL)
geschaeft2004_541.htm
Niklaus Scherr (AL)
geschaeft2020_17.htm
Hans Jörg Käppeli (SP)
geschaeft1998_191.htm
Rolf Walther (FDP)
geschaeft2011_438.htm
Linda Bär (SP)
geschaeft1998_59.htm
Christoph Spiess (SD)
geschaeft2017_455.htm
Katharina Prelicz-Huber (Grüne)
geschaeft2007_327.htm
Walter Angst (AL)
geschaeft2007_469.htm
Peter Anderegg (EVP)
geschaeft2001_389.htm
Renate Schoch (AL)
geschaeft2005_81.htm
Salvatore Di Concilio (SP)
geschaeft2015_236.htm
Raphael Kobler (FDP)
geschaeft2017_9.htm
Andreas Egli (FDP)
geschaeft2018_215.htm
Luca Maggi (Grüne)
geschaeft2011_148.htm
geschaeft1999_180.htm
Peter Marti ()
geschaeft2016_450.htm
Hans Jörg Käppeli (SP)
geschaeft1998_378.htm
Oliver B. Meier (SVP)
geschaeft2016_322.htm
Ursula Näf (SP)
geschaeft2017_131.htm
Gabriele Kisker (Grün

geschaeft2013_30.htm
Helen Glaser (SP)
geschaeft2004_184.htm
Balthasar Glättli (Grüne)
geschaeft2013_305.htm
Niklaus Scherr (AL)
geschaeft2000_58.htm
Marcel Savarioud (SP)
geschaeft2006_240.htm
Yvonne Müller (SP)
geschaeft2013_24.htm
Cäcilia Hänni-Etter (FDP)
geschaeft2000_239.htm
Markus Schwyn (PFZ)
geschaeft2009_83.htm
Michael Baumer (FDP)
geschaeft2005_432.htm
Roger Bartholdi (SVP)
geschaeft2008_405.htm
Markus Knauss (Grüne)
geschaeft2019_164.htm
Mischa Schiwow (AL)
geschaeft2018_363.htm
Luca Maggi (Grüne)
geschaeft2008_377.htm
Markus Schwyn (PFZ)
geschaeft2015_354.htm
Walter Angst (AL)
geschaeft2018_405.htm
Barbara Wiesmann (SP)
geschaeft1999_155.htm
Thomas Marthaler (SP)
geschaeft2016_14.htm
Samuel Balsiger (SVP)
geschaeft2006_446.htm
Monika Erfigen (SVP)
geschaeft2006_320.htm
Alexander Jäger (FDP)
geschaeft2016_452.htm
Daniel Regli (SVP)
geschaeft2013_271.htm
Marco Denoth (SP)
geschaeft2005_54.htm
Balthasar Glättli (Grüne)
geschaeft2010_359.htm
Roland Scheck (SVP)
geschaeft2008_5

geschaeft2004_183.htm
geschaeft2016_253.htm
Ann-Catherine Nabholz (GLP)
geschaeft2005_179.htm
Roger Bartholdi (SVP)
geschaeft2005_151.htm
Bernhard Piller (Grüne)
geschaeft2015_145.htm
Markus Kunz (Grüne)
geschaeft2009_375.htm
Thomas Schwendener (SVP)
geschaeft2017_84.htm
Balz Bürgisser (Grüne)
geschaeft2009_361.htm
Walter Angst (AL)
geschaeft2014_342.htm
Matthias Probst (Grüne)
geschaeft2004_430.htm
Gregor Bucher ()
geschaeft2019_375.htm
Gabriele Kisker (Grüne)
geschaeft2004_13.htm
Susi Gut (PFZ)
geschaeft2007_518.htm
Balthasar Glättli (Grüne)
geschaeft2019_56.htm
Elisabeth Schoch (FDP)
geschaeft2018_199.htm
Stefan Urech (SVP)
geschaeft2017_256.htm
Andreas Egli (FDP)
geschaeft2015_186.htm
Hans Jörg Käppeli (SP)
geschaeft2014_395.htm
Elisabeth Schoch (FDP)
geschaeft2003_100.htm
Emil Seliner (SP)
geschaeft2007_524.htm
Roger Bartholdi (SVP)
geschaeft2017_47.htm
Andreas Egli (FDP)
geschaeft2007_256.htm
Markus Schwyn (PFZ)
geschaeft2017_46.htm
Derek Richter (SVP)
geschaeft2007_257.htm
Marku

geschaeft2008_303.htm
Ruth Ackermann (CVP)
geschaeft2018_471.htm
Marcel Savarioud (SP)
geschaeft2010_271.htm
Daniel Meier (CVP)
geschaeft2003_239.htm
Thomas  Kappeler (CVP)
geschaeft2016_74.htm
Felix Moser (Grüne)
geschaeft2016_368.htm
Daniel Regli (SVP)
geschaeft2000_488.htm
Monika Erfigen (SVP)
geschaeft2013_239.htm
Gerhard Bosshard (EVP)
geschaeft2005_34.htm
Theo Hauri (SVP)
geschaeft2006_354.htm
Rolf Kuhn (SP)
geschaeft2013_211.htm
Roland Scheck (SVP)
geschaeft2018_65.htm
Michael Kraft (SP)
geschaeft2006_426.htm
Roger Tognella (FDP)
geschaeft2017_147.htm
Walter Anken (SVP)
geschaeft2015_268.htm
Simone Brander (SP)
geschaeft2011_102.htm
Roland Scheck (SVP)
geschaeft2011_82.htm
Michel Urben (SP)
geschaeft2008_505.htm
Susi Gut (PFZ)
geschaeft2006_397.htm
Patrick Blöchlinger (SD)
geschaeft2005_532.htm
Kurt Maeder (CVP)
geschaeft2017_190.htm
Martin Götzl (SVP)
geschaeft2006_383.htm
Mauro Tuena (SVP)
geschaeft2018_263.htm
Pirmin Meyer (GLP)
geschaeft2007_190.htm
Bruno Sidler (SVP)
gescha

geschaeft2002_159.htm
Roger Bartholdi (SVP)
geschaeft2014_62.htm
Walter Angst (AL)
geschaeft2006_227.htm
Roger Liebi (SVP)
geschaeft2007_36.htm
Markus Knauss (Grüne)
geschaeft2013_410.htm
Urs Egger (FDP)
geschaeft2005_125.htm
Rolf Kuhn (SP)
geschaeft2007_4.htm
Susi Gut (PFZ)
geschaeft2004_322.htm
Bruno Amacker (SVP)
geschaeft2018_112.htm
Jean-Daniel Strub (SP)
geschaeft2009_473.htm
Rolf Stucker (SVP)
geschaeft2011_267.htm
Severin Pflüger (FDP)
geschaeft2009_315.htm
Markus Schwyn (PFZ)
geschaeft2007_587.htm
Mirella Wepf (SP)
geschaeft2001_273.htm
Köbi Möri (Parteilos)
geschaeft2010_12.htm
Urs Weiss (SVP)
geschaeft2011_298.htm
Ursula Uttinger (FDP)
geschaeft2019_22.htm
Markus Kunz (Grüne)
geschaeft2003_174.htm
Ruth Anhorn (SVP)
geschaeft2003_52.htm
Markus Schwyn (PFZ)
geschaeft2007_544.htm
Markus Knauss (Grüne)
geschaeft2017_32.htm
Christina Schiller (AL)
geschaeft2007_545.htm
Susi Gut (PFZ)
geschaeft2002_372.htm
Balthasar Glättli (Grüne)
geschaeft2003_47.htm
Uschi Heinrich (SP)
geschaef

KeyboardInterrupt: 

### Vorarbeiten 2 für den Loop: Nur Geschäfte mit mehreren Einreichenden rausfiltern:

In [ ]:
files = os.listdir("motionen")

for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("motionen/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #So filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
             
            #print (file) #damit finden wir raus, wo es allenfalls Probleme gibt
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                #print(einreichender1)
                
                #try:
                    #einreichender2 = einreichende[1].text.strip()
                    #print(einreichender2)
                
                #except: 
                    #einreichender2 = ""
                        
                try:
                    einreichender3 = einreichende[2].text.strip()
                    print(einreichender3)
                except: 
                    einreichender3 = ""

So haben wir rausgefunden: Es gibt keine Geschäfte, die von mehr als 2 Personen eingereicht wurden – das vereinfacht die Sache wesentlich: Wir können also eine Liste machen, die durchs Band zwei Personen umfasst.

### Vorarbeiten 3 für den Loop: Jetzt holen wir alle gesuchten Daten raus

In [23]:
from tqdm import tqdm

In [24]:
files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #Jetzt filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                
                name1 = einreichender1.split(" (")[0]
                partei1 = einreichender1.split(" (")[1][:-1]
                
                einreichender2 = einreichende[1].text.strip()
                name2 = einreichender2.split(" (")[0]
                partei2 = einreichender2.split(" (")[1][:-1]
                
                geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                #datum = soup.find_all("div")[31].find_all("strong")[1].text.split(" vom ")[1][:-1].strip()
                try: 
                    departement = soup.find_all("div")[28].find_all("span")[2].text
                except: 
                    departement = ""
                            
                #Zum Schluss: Wir wollen nur jene in unserem Dict mit unterschiedlichen Parteien
                if partei1 != partei2:
                    mini_dict = {
                        #"Datum" : datum,
                        "Geschäft" : geschaeft_nr,
                        "Einreichender 1" : name1,
                        "Partei 1" : partei1,
                        "Einreichender 2" : name2,
                        "Partei 2" : partei2,
                        "Departement" : departement
                    }
                    df_list.append(mini_dict)  

  3%|▎         | 231/7026 [00:07<03:53, 29.16it/s]


KeyboardInterrupt: 

Beim Datum und Departement funktioniert was nicht, sonst läufts!

### >>> Mit einfachem Loop testen, wo Datum rauslesen nicht funktioniert:

In [25]:
files = os.listdir("alle_geschaefte")

for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #So filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
             
            print(file)
            
            try:
                datum = soup.find_all("div")[31].find_all("strong")[0].text.split(" vom ")[1][:-1].strip()
            except:
                try: 
                    datum = soup.find_all("div")[31].find_all("strong")[1].text.split(" vom ")[1][:-1].strip()
                except:
                    datum = soup.find_all("div")[31].find_all("td")[3].find_all("p")[1].text.split(" vom ")[1][:-1].strip()  
            
            print(datum)

geschaeft2001_362.htm
27.6.2001
geschaeft2000_171.htm
5.4.2000
geschaeft2012_99.htm
14.03.2012
geschaeft2014_227.htm
02.07.2014
geschaeft2011_376.htm


IndexError: list index out of range

In [26]:
# Jetzt testen wir die fehlerhaften File:
file = open("alle_geschaefte/geschaeft2012_99.htm", "r")
text = file.read()
soup = BeautifulSoup(text, "html.parser")

#Erkenntnis: Das Datim ist manchmal nicht in folgendem Fragment:
#soup.find_all("div")[31].find_all("strong")[1].text
#sondern in diesem hier:
#soup.find_all("div")[31].find_all("strong")[0].text
#Fazit: es braucht hier eine try-funktion

#manchmal findet man es so: (Ist das eventuell der besser Code für alle? > testen!)
soup.find_all("div")[31].find_all("td")[3].find_all("p")[1].text.split(" vom ")[1][:-1].strip()  

# Fazit: Geht, wird aber zu mühsam. Ich versuche es stttdessen mir Regex in den Griff zu bekommen

'14.03.2012'

In [27]:
import re

file = open("alle_geschaefte/geschaeft2004_406.htm", "r")
text = file.read()
soup = BeautifulSoup(text, "html.parser")

#wir mussten hier nachträglich noch eine Wihtespace-Putzfunktion einbauen, weils vor dem Jahr manchmal einen hat
regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
datum = re.findall(regex, str(soup))[0].replace(" ", "")
datum

'18.8.2004'

In [28]:
#Das funktioniert, darum testen wir es jetzt im Loop mit allen files:

files = os.listdir("alle_geschaefte")

for file in files: 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #So filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
             
            print(file)
            
            regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
            datum = re.findall(regex, str(soup))[0].replace(" ", "")

            print(datum)

geschaeft2001_362.htm
27.6.2001
geschaeft2000_171.htm
5.4.2000
geschaeft2012_99.htm
14.03.2012
geschaeft2014_227.htm
02.07.2014
geschaeft2011_376.htm
05.10.2011
geschaeft2009_562.htm
25.11.2009
geschaeft2000_617.htm
20.12.2000
geschaeft2015_84.htm
18.03.2015
geschaeft2020_2.htm
08.01.2020
geschaeft2011_362.htm
28.09.2011
geschaeft2004_541.htm
20.10.2004
geschaeft2009_238.htm
03.06.2009
geschaeft1998_191.htm
29.01.20
geschaeft2002_289.htm
21.8.2002
geschaeft2000_159.htm
29.3.2000
geschaeft2011_438.htm
23.11.2011
geschaeft2017_327.htm
20.09.2017
geschaeft2016_134.htm
20.04.2016
geschaeft2007_455.htm
22.8.2007
geschaeft2017_455.htm
13.12.2017
geschaeft2012_262.htm
20.06.2012
geschaeft2006_134.htm
3.5.2006
geschaeft2002_276.htm
21.8.2002
geschaeft2008_16.htm
26.05.2010
geschaeft2007_469.htm
29.8.2007
geschaeft2012_66.htm
29.02.2012
geschaeft1998_146.htm
13.5.1998
geschaeft2000_415.htm
30.8.2000
geschaeft2004_3.htm
7.1.2004
geschaeft2005_81.htm
2.3.2005
geschaeft2015_236.htm
01.07.2015
gesc

geschaeft2011_349.htm
21.09.2011
geschaeft2020_14.htm
15.01.2020
geschaeft2008_15.htm
16.01.2008
geschaeft2017_330.htm
20.09.2017
geschaeft2012_261.htm
20.06.2012
geschaeft2006_137.htm
3.5.2006
geschaeft2015_44.htm
04.02.2015
geschaeft2007_324.htm
06.06.2007
geschaeft1998_179.htm
10.6.1998
geschaeft2007_330.htm
6.6.2007
geschaeft2016_137.htm
20.04.2016
geschaeft2007_456.htm
22.8.2007
geschaeft2012_71.htm
29.02.2012
geschaeft1991_56.htm
30.1.91
geschaeft2012_249.htm
13.06.2012
geschaeft2017_318.htm
13.09.2017
geschaeft2006_10.htm
18.1.2006
geschaeft2018_216.htm
06.06.2018
geschaeft2005_96.htm
16.3.2005
geschaeft2000_364.htm
12.7.2000
geschaeft2010_416.htm
29.09.2010
geschaeft2015_235.htm
01.07.2015
geschaeft2005_82.htm
2.3.2005
geschaeft2011_177.htm
29.01.20
geschaeft2005_209.htm
25.5.2005
geschaeft2008_558.htm
10.12.2008
geschaeft2010_358.htm
01.09.2010
geschaeft2003_502.htm
17.12.2003
geschaeft2011_20.htm
19.01.2011
geschaeft2016_29.htm
20.01.2016
geschaeft2017_126.htm
10.05.2017
gesc

KeyboardInterrupt: 

### Heureka! Also: Zweiter Anlauf kompletter Loop:

In [29]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re

files = os.listdir("alle_geschaefte")
df_list = []

for file in tqdm(files): 
    
    if "geschaeft" in file: #um fehlerhafte files auszuschliessen
        geschaeft = open("alle_geschaefte/"+ file, "r", encoding = "utf-8")
        text = geschaeft.read()
        soup = BeautifulSoup(text, "html.parser")
    
        #Jetzt filtern wir nur jene raus, die mehrere Einreichende haben:
        einreichende = soup.find_all("div")[24].find_all("a")
        if len(einreichende) > 1: 
            
            #Jetzt filtern wir jene, die überhaupt einen Namen drin haben; Erkennungsmerkmal: Klammern
            einreichender1 = einreichende[0].text.strip()
            if "(" and ")" in einreichender1:
                
                name1 = einreichender1.split(" (")[0]
                partei1 = einreichender1.split(" (")[1][:-1]
                
                einreichender2 = einreichende[1].text.strip()
                name2 = einreichender2.split(" (")[0]
                partei2 = einreichender2.split(" (")[1][:-1]
                
                geschaeft_nr = soup.find_all("div")[22].find_all("span")[1].text
                
                regex = r"((?<=\svom\s)\d*\.\d*\.\s*\d*)"
                datum = re.findall(regex, str(soup))[0].replace(" ", "")
                
                try: 
                    regex_dep = r"((?<=Zuständiges Departement).*\n*.*)"
                    dep_fragment = re.findall(regex_dep, str(soup))[0]
                    soup_dep = BeautifulSoup(dep_fragment, "html.parser")
                    departement = soup_dep.text[1:]

                except: 
                    departement = ""
                            
                #Zum Schluss: Wir wollen nur jene in unserem Dict mit unterschiedlichen Parteien
                if partei1 != partei2:
                    mini_dict = {
                        "Datum" : datum,
                        "Geschäft" : geschaeft_nr,
                        "Einreichender 1" : name1,
                        "Partei 1" : partei1,
                        "Einreichender 2" : name2,
                        "Partei 2" : partei2,
                        "Departement" : departement
                    }
                    df_list.append(mini_dict)  

100%|██████████| 7026/7026 [05:22<00:00, 21.76it/s]


In [ ]:
len(df_list)

In [4]:
df = pd.DataFrame(df_list)
df.head(10)

,Datum,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
0,25.11.2009,2009/562,Kurt Hüssy,SVP,Peter Anderegg,EVP,Polizeidepartement (PD)
1,08.01.2020,2020/2,Marcel Bührig,Grüne,Natascha Wey,SP,Präsidialdepartement (PRD)
2,28.09.2011,2011/362,Alecs Recher,AL,Maleica Landolt,GLP,Finanzdepartement (FD)
3,20.10.2004,2004/541,Niklaus Scherr,AL,Kurt Maeder,CVP,Hochbaudepartement (HBD)
4,29.01.20,1998/191,Rolf Walther,FDP,Jürg Liebermann,,Finanzdepartement (FD)
5,21.8.2002,2002/289,Markus Schwyn,PFZ,Monika Erfigen,SVP,Hochbaudepartement (HBD)
6,23.11.2011,2011/438,Linda Bär,SP,Matthias Probst,Grüne,Präsidialdepartement (PRD)
7,3.5.2006,2006/134,Richard Rabelbauer,EVP,Robert Schönbächler,CVP,Departement der Industriellen Betriebe (DIB)
8,26.05.2010,2008/16,Rebekka Wyler,SP,Pierino Cerliani,Grüne,Finanzdepartement (FD)
9,29.8.2007,2007/469,Peter Anderegg,EVP,Daniel Meier,CVP,Polizeidepartement (PD)


In [367]:
df.to_csv("Dataframe_komplett.csv")

### Kontrolle und Datenbereinigung

In [5]:
df["Partei 1"].value_counts().head(20)

SP           283
Grüne        204
FDP          130
PFZ          112
AL            99
SVP           92
CVP           91
              57
GLP           54
EVP           40
Parteilos     14
CSP           11
SL             3
FraP           3
32 KB          1
LdU            1
85 KB          1
Name: Partei 1, dtype: int64

In [6]:
# Wer sind diese Leute, die nicht als Parteilos vermerkt sind, sondern gar keine haben?
df[df["Partei 1"] == ""]["Einreichender 1"].value_counts()

Gregor Bucher          18
Andreas Ammann         10
Andreas J. Schlegel     6
Urs Rechsteiner         6
Peter Marti             4
Jürg Liebermann         4
Fakir Atalay            4
Placidus Maissen        2
Anton Stäbler           2
Salomon Browar          1
Name: Einreichender 1, dtype: int64

In [7]:
# Jetzt machen wir zum Reaprieren ein Dataframe, in dem wir den Namen die Parteien zuordnen:

Reparatur_dict = {"Gregor Bucher":"Grüne",
                  "Andreas Ammann":"SP",
                  "Urs Rechsteiner":"CVP",
                  "Andreas J. Schlegel":"FDP",
                  "Jürg Liebermann":"FDP",
                  "Peter Marti":"FDP",
                  "Fakir Atalay":"SP",
                  "Anton Stäbler":"CVP",
                  "Placidus Maissen":"CVP",
                  "Salomon Browar":"Grüne",
                  "Monika Piesbergen":"FDP",
                  "Hanna Lienhard":"FDP",
                  "Urs Lauffer":"FDP",
                  "Esther Ponti-Weder":"CVP"}

In [8]:
#Einer von vielen misslungenen Ansätzen:
def Reparatur(Person):
    if Person in Reparatur_dict:
        return Reparatur_dict[Person]

 df["Partei 1"] = df["Einreichender 1"].apply(Reparatur)
#geht nicht, weil es so die Werte aller anderen mit none überschreibt

IndentationError: unexpected indent (<ipython-input-8-11c97e9abe3e>, line 2)

In [371]:
#So steuern wir eine leere Partei an, deren Position wir kennen:
df.loc[41, 'Einreichender 1']

'Jürg Liebermann'

In [10]:
#Jetzt machen wir eine Liste, in der der Index aller fehlerhaften Positionen für den Einreichenden 1 enthalten sind:
fehlerposition1 = df[df["Partei 1"] == ""]["Einreichender 1"].index.tolist()

In [11]:
def Reparatur(Person):
    return Reparatur_dict[Person]

#Hier steuern wir nn für jede fehlerhafte Zeile die (leere) Partei-Zelle an, und füllen diese mit dem richtigen Wert.
#Dazu ersetzen wir die Leerstelle "" mit dem korekten Wert.
#Diesen erhalten wir, indem wir den Namen der gleichen Zeile in unsere Funktion schicken, 
#die aus dem Reparatur-Dict die zugehörige Partei holt.
for zeile in fehlerposition1:
    df.loc[zeile, 'Partei 1'] = df.loc[zeile, 'Partei 1'].replace("", Reparatur(df.loc[zeile, "Einreichender 1"]))

In [12]:
# Jetzt testen wir das – Heureka!
df[df["Einreichender 1"] == "Gregor Bucher"]

,Datum,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
108,18.5.2005,2005/194,Gregor Bucher,Grüne,Daniel Leupi,Grüne,Hochbaudepartement (HBD)
164,15.11.2006,2006/495,Gregor Bucher,Grüne,Ernst Danner,EVP,Präsidialdepartement (PRD)
203,18.5.2005,2005/190,Gregor Bucher,Grüne,Muriel Herzig,Grüne,Sozialdepartement (SD)
267,18.5.2005,2005/192,Gregor Bucher,Grüne,Eva Virag Jansen,Grüne,Hochbaudepartement (HBD)
271,18.5.2005,2005/193,Gregor Bucher,Grüne,Pierino Cerliani,Grüne,Hochbaudepartement (HBD)
316,08.01.20,2007/344,Gregor Bucher,Grüne,Ernst Danner,EVP,Präsidialdepartement (PRD)
355,1.9.2004,2004/457,Gregor Bucher,Grüne,Daniel Leupi,Grüne,Schul- und Sportdepartement (SSD)
550,16.6.2004,2004/321,Gregor Bucher,Grüne,Walter Angst,AL,Schul- und Sportdepartement (SSD)
612,08.01.20,2006/406,Gregor Bucher,Grüne,Muriel Herzig,Grüne,Sozialdepartement (SD)
726,22.9.2004,2004/515,Gregor Bucher,Grüne,Pierino Cerliani,Grüne,Präsidialdepartement (PRD)


In [13]:
df["Partei 1"].value_counts()

SP           297
Grüne        223
FDP          144
PFZ          112
CVP          101
AL            99
SVP           92
GLP           54
EVP           40
Parteilos     14
CSP           11
FraP           3
SL             3
85 KB          1
32 KB          1
LdU            1
Name: Partei 1, dtype: int64

In [14]:
#Weils geklappt hat,machen wir hier das gleiche noch für die Einreichenden 2:
def Reparatur(Person):
    return Reparatur_dict[Person]

fehlerposition2 = df[df["Partei 2"] == ""]["Einreichender 2"].index.tolist()

for zeile in fehlerposition2:
    df.loc[zeile, 'Partei 2'] = df.loc[zeile, 'Partei 2'].replace("", Reparatur(df.loc[zeile, "Einreichender 2"]))

Achtung, durch die Reparatur sind möglicherweise Paare der gleichen Partei oder des gleichen Blocks entstanden, die müssen wir rausfiltern:

In [15]:
#Hier ist die Variante für Parteien:
for zeile in range(0, len(df)-1):
    if df.loc[zeile, "Partei 1"] == df.loc[zeile, "Partei 2"]:
        df.drop(zeile, inplace = True)

In [33]:
len(df)

1115

In [17]:
#Weil wir endlich am Ziel sind, speichern wir den bereinigten Dataframe:
df.to_csv("Dataframe_bereinigt_parteiuebergreifend.csv")

### Hier werten wir die Zahlen nach Legislatur aus (davor die Daten säubern)

bei diesen Positionen stimmt beim Datum etwas Grundsätzliches nicht, weshalb wir es mit einem annäherungsweisen Datum aufgrund der Geschäftnummer ersetzen:

In [35]:
fehlerliste = ["29.01.20","08.01.20","18.12.19"]

for fehler in fehlerliste:
    fehlerposition = df[df["Datum"] == fehler].index.tolist()
    for zeile in fehlerposition:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        Ersatzdatum = "31.12." + Jahr
        df.loc[zeile, "Datum"] = df.loc[zeile, "Datum"].replace(fehler, Ersatzdatum)

Hier korrigieren wir zwei weitere Fehler, bei denen die Daten zwar stimmen, aber falsch formatiert sind (Leerschläge, zweistellige Jahreszahlen):

In [36]:
df.head(5)

,Datum,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
0,25.11.2009,2009/562,Kurt Hüssy,SVP,Peter Anderegg,EVP,Polizeidepartement (PD)
1,08.01.2020,2020/2,Marcel Bührig,Grüne,Natascha Wey,SP,Präsidialdepartement (PRD)
2,28.09.2011,2011/362,Alecs Recher,AL,Maleica Landolt,GLP,Finanzdepartement (FD)
3,20.10.2004,2004/541,Niklaus Scherr,AL,Kurt Maeder,CVP,Hochbaudepartement (HBD)
5,21.8.2002,2002/289,Markus Schwyn,PFZ,Monika Erfigen,SVP,Hochbaudepartement (HBD)


In [37]:
len(df)

1115

In [54]:
df_laenge = len(df)
for zeile in range(0, df_laenge-1):
    
    #hier holen wir jene Daten raus, die nur eine zweistellige als Jahreszahl haben und vervollständigen sie:
    if df.loc[zeile,"Datum"][-3:-2] == ".":
        if df.loc[zeile,"Datum"][-2:] >= "80":
            df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-2]+"19"+df.loc[zeile,"Datum"][-2:]
        if df.loc[zeile,"Datum"][-2:] <= "21":
            df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-2]+"20"+df.loc[zeile,"Datum"][-2:] 
    
    #hier holen wir jene Daten raus, die einen Leerschlag drin haben und löschen ihn raus
    if df.loc[zeile,"Datum"][-3:-2] == " ":
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-3]+df.loc[zeile,"Datum"][-2:]
        
    #hier holen wir jene Daten mit vierer-Jahreszahlen raus, die einen Leerschlag davor haben und löschen ihn raus
    if df.loc[zeile,"Datum"][-5:-4] == " ":
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"][:-5]+df.loc[zeile,"Datum"][-4:]
    
    #Hier holen wir jene raus, die zu kurz sind, weil die Jahreszahl ganz fehlt:
    if len(df.loc[zeile,"Datum"]) == 6:
        Jahr = df.loc[zeile, "Geschäft"].split("/")[0]
        df.loc[zeile,"Datum"] = df.loc[zeile,"Datum"]+Jahr
        
        print(df.loc[zeile,"Datum"])

KeyError: 4

In [55]:
df["Datum"] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')
df = df.set_index("Datum")
df.head(5)

,Geschäft,Einreichender 1,Partei 1,Einreichender 2,Partei 2,Departement
Datum,,,,,,
2009-11-25,2009/562,Kurt Hüssy,SVP,Peter Anderegg,EVP,Polizeidepartement (PD)
2020-01-08,2020/2,Marcel Bührig,Grüne,Natascha Wey,SP,Präsidialdepartement (PRD)
2011-09-28,2011/362,Alecs Recher,AL,Maleica Landolt,GLP,Finanzdepartement (FD)
2004-10-20,2004/541,Niklaus Scherr,AL,Kurt Maeder,CVP,Hochbaudepartement (HBD)
2002-08-21,2002/289,Markus Schwyn,PFZ,Monika Erfigen,SVP,Hochbaudepartement (HBD)


Jetzt messen wir für die Statistik die Zahl pro Legislatur:

In [56]:
df_1990 = df["1990-05-15":"1994-05-14"]
df_1994 = df["1994-05-15":"1998-05-14"]
df_1998 = df["1998-05-15":"2002-05-14"]
df_2002 = df["2002-05-15":"2006-05-14"]
df_2006 = df["2006-05-15":"2010-05-14"]
df_2010 = df["2010-05-15":"2014-05-14"]
df_2014 = df["2014-05-15":"2018-05-14"]
df_2018 = df["2018-05-15":"2022-05-14"]

print("1990-94: "+str(len(df_1990)))
print("1994-98: "+str(len(df_1994)))
print("1998-02: "+str(len(df_1998)))
print("2002-06: "+str(len(df_2002)))
print("2006-10: "+str(len(df_2006)))
print("2010-14: "+str(len(df_2010)))
print("2014-18: "+str(len(df_2014)))
print("2018-22: "+str(len(df_2018)))

1990-94: 3
1994-98: 16
1998-02: 133
2002-06: 242
2006-10: 233
2010-14: 162
2014-18: 197
2018-22: 129
